### 1) Function folding and application

In [2]:
(<$<) :: (a -> b) -> a -> b
(<$<) = ($)

(>$>) :: a -> (a -> b) -> b
x >$> f = f x
infixl 0 >$>

(<.<) :: (b -> c) -> (a -> b) -> (a -> c)
(<.<) = (.)

(>.>) :: (a -> b) -> (b -> c) -> (a -> c)
f >.> g = g . f
infixl 9 >.>

safeTail :: [a] -> Maybe [a]
safeTail [] = Nothing
safeTail (x:xs) = Just xs

extractMaybe :: Maybe a -> a
extractMaybe Nothing = error "Nothing inside!"
extractMaybe (Just x) = x

insertMaybe :: a -> Maybe a
insertMaybe = Just

(>^$>) :: Maybe a -> (a -> Maybe b) -> Maybe b
Nothing >^$> _ = Nothing
(Just x) >^$> f = f x
infixl 1 >^$>

f1 :: (Ord a, Num a) => a -> Maybe a
f1 x = if x > 0 then Just (x + 1) else Nothing

f2 :: (Eq a, Num a) => a -> Maybe a
f2 x = if x /= 0 then Just (10 * x) else Nothing

(>.>>) :: (a -> Maybe b) -> (b -> Maybe c) -> (a -> Maybe c)
f >.>> g = \x -> g (extractMaybe (f x))

Line 40: Avoid lambda
Found:
\ x -> g (extractMaybe (f x))
Why not:
g . extractMaybe . f

In [4]:
(>.>>) :: (a -> Maybe b) -> (b -> Maybe c) -> (a -> Maybe c)
f >.>> g = \x -> (f x) >^$> g


Line 2: Redundant bracket
Found:
(f x) >^$> g
Why not:
f x >^$> g

### 2) Maybe monad

In [11]:
import Control.Monad
:t (>=>)

:t (>>=)

join :: Maybe (Maybe a) -> Maybe a
join Nothing = Nothing
join (Just Nothing) = Nothing
join (Just (Just x)) = Just x

insertMaybe [1,2]
return (Just [1,2])
return [1,2]
return [1,2] :: Maybe [Int]

insertMaybe [1,2] >^$> safeTail
return [1,2] >>= safeTail
(return [1,2] >>= safeTail) == (insertMaybe [1,2] >^$> safeTail)

insertMaybe [1,2] >^$> safeTail >^$> safeTail >^$> safeTail >^$> safeTail
return [1,2] >>= safeTail >>= safeTail >>= safeTail >>= safeTail

return [1,2] >^$> safeTail >^$> safeTail >^$> safeTail >^$> safeTail
insertMaybe [1,2] >>= safeTail >^$> safeTail >>= safeTail >^$> safeTail
return [1,2] >^$> safeTail >>= safeTail >^$> safeTail >>= safeTail

doSafeTail3x :: [a] -> Maybe [a]
doSafeTail3x xs = do
  t1 <- safeTail xs
  t2 <- safeTail t1
  t3 <- safeTail t2
  return t3

safeTail3x :: [a] -> Maybe [a]
safeTail3x xs =
  safeTail xs >>= \t1 ->
    safeTail t1 >>= \t2 ->
      safeTail t2 >>= \t3 ->
        return t3

safeTail3x' :: [a] -> Maybe [a]
safeTail3x' xs = return xs >>= safeTail >>= safeTail >>= safeTail

doSafeTail3x [1..5]
safeTail3x [1..5]
safeTail3x' [1..5]

doSafeTail3x [1]
safeTail3x [1]
safeTail3x' [1]

f5 :: Int -> Int -> Int -> Int
f5 x y z = 1000 `div` x + 100 `div` y + 10 `div` z

f5 1 2 3
f5 1 2 0
f5 1 0 3
f5 0 2 3

safeDiv :: Int -> Int -> Maybe Int
safeDiv x y | y /= 0    = Just $ x `div` y
            | otherwise = Nothing

safeF5 :: Int -> Int -> Int -> Maybe Int
safeF5 x y z =
  case (safeDiv 1000 x) of
    Nothing -> Nothing
    Just (iOverX) ->
      case (safeDiv 100 y) of
        Nothing -> Nothing
        Just (iOverY) ->
          case (safeDiv 10 z) of
            Nothing -> Nothing
            Just (iOverZ) -> Just $ iOverX + iOverY + iOverZ
            
safeF5 1 2 3
safeF5 1 2 0
safeF5 1 0 3
safeF5 0 2 3

safeF5' :: Int -> Int -> Int -> Maybe Int
safeF5' x y z = do
  iOverX <- safeDiv 1000 x
  iOverY <- safeDiv 100 y
  iOverZ <- safeDiv 10 z
  return $ iOverX + iOverY + iOverZ
  
safeF5'' :: Int -> Int -> Int -> Maybe Int
safeF5'' x y z = f <$> iOverX <*> iOverY <*> iOverZ
  where
    f i j k = i + j + k
    iOverX = safeDiv 1000 x
    iOverY = safeDiv 100 y
    iOverZ = safeDiv 10 z

(>=>) :: forall (m :: * -> *) a b c. Monad m => (a -> m b) -> (b -> m c) -> a -> m c

(>>=) :: forall (m :: * -> *) a b. Monad m => m a -> (a -> m b) -> m b

Just [1,2]

Just [1,2]

[1,2]

...

Nothing

Nothing

Nothing

1053

: 

### 4) [] monad

In [12]:
:i []

[1,2,3] :: [Int]
[1,2,3] :: [] Int

return 1 :: [] Int
return 3 >>= (\x -> [1..x])
[1,2] >>= (\x -> [-x,x])
[1,2,3] >>= (\x -> [-x..x])

[1,2] >>= (\x -> [-x,x]) >>= (\y -> [-y,y])
[1,2] >>= \x -> [-x,x] >>= \y -> [-y,y]
[1,2] >>= \x -> [-x,x] >>= \y -> return (x,y)

xs1 :: [(Int,Int,Int)]
xs1 = [ (x,y,z) | let xs = [1,2], x <- xs, y <- xs, z <- xs ]

doXs1 :: [(Int,Int,Int)]
doXs1 = do
  let xs = [1,2]
  x <- xs
  y <- xs
  z <- xs
  return (x,y,z)
  
xs1
doXs1
xs1 == doXs1  
  

import Control.Monad

xs2 :: [(Int,Int,Int)]
xs2 = [ (x,y,z) | let xs = [1..3], x <- xs, y <- xs, z <- xs, x > y && y > z ]

doXs2 :: [(Int,Int,Int)]
doXs2 = do
  let xs = [1..3]
  x <- xs
  y <- xs
  z <- xs
  guard $ x > y && y > z
  return (x,y,z)

doXs2' :: [(Int,Int,Int)]
doXs2' = do
  let xs = [1..3]
  x <- xs
  y <- xs
  z <- xs
  if x > y && y > z
    then return (x,y,z)
    else []
    
:t guard
[1..10] >>= \x -> guard (even x) >> return x
[1..10] >>= \x -> guard (even x) >>= \y -> return (x,y)

[1..10] >>= \x -> return x
[1..10] >>= return

join :: [[a]] -> [a]
join [] = []
join (x:xs) = x ++ join xs

[1,2,3]

[1,2,3]

[1]

[1,2,3]

[-1,1,-2,2]

...

guard :: forall (f :: * -> *). Alternative f => Bool -> f ()

[2,4,6,8,10]

[(2,()),(4,()),(6,()),(8,()),(10,())]

[1,2,3,4,5,6,7,8,9,10]

[1,2,3,4,5,6,7,8,9,10]

### 5) Writer monad

In [17]:
-- :i Writer

import Control.Monad.Trans.Writer.Lazy
:i Writer
:i WriterT

:t writer
:t runWriter
:t execWriter
:t tell

runWriter $ writer(1,["op1","op2"])
execWriter $ writer(1,["op1","op2"])

runWriter (return 1 :: Writer String Int)
runWriter (return [1] :: Writer String [Int])
runWriter (return (1,1) :: Writer [String] (Int,Int))

-- import Control.Monad.Trans.Writer.Lazy
"gcdWithLog"
gcdWithLog :: Int -> Int -> Writer [String] Int
gcdWithLog a b
  | b == 0 = do
      tell ["Finished with " ++ show a]
      return a
  | otherwise = do
      tell [show a ++ " mod " ++ show b ++ " = " ++ show (a `mod` b)]
      gcdWithLog b (a `mod` b)
      
gcdWithLog 4 16
runWriter $ gcdWithLog 4 16
execWriter $ gcdWithLog 4 16

"mapWithLog"
mapWithLog :: Show a => (a -> b) -> [a] -> Writer [String] [b]
mapWithLog _ [] = do
  tell ["map []"]
  return []
mapWithLog f (x:xs) = do
  tell ["map " ++ show x]
  mapXs <- mapWithLog f xs
  return $ f x : mapXs

mapWithLog (*2) [1..5]
runWriter $ mapWithLog (*2) [1..5]
execWriter $ mapWithLog (*2) [1..7]

writer :: forall (m :: * -> *) a w. Monad m => (a, w) -> WriterT w m a

runWriter :: forall w a. Writer w a -> (a, w)

execWriter :: forall w a. Writer w a -> w

tell :: forall (m :: * -> *) w. Monad m => w -> WriterT w m ()

(1,["op1","op2"])

...

["4 mod 16 = 4","16 mod 4 = 0","Finished with 4"]

"mapWithLog"

WriterT (Identity ([2,4,6,8,10],["map 1","map 2","map 3","map 4","map 5","map []"]))

([2,4,6,8,10],["map 1","map 2","map 3","map 4","map 5","map []"])

["map 1","map 2","map 3","map 4","map 5","map 6","map 7","map []"]